In [1]:
from codebase.classes import Data, Particles
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp




## Big 5 EZ

In [2]:
existing_directory = None
task_handle = 'big5_ez'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201205_201808_big5/


In [3]:

# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 'big5', 
    size = 50,
    random_seed = 0
    )
    
exp_data.generate()





Reading data for women


N = 677, J= 15, K =5


In [4]:

model_num = 1
## setup particles
param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
particles = Particles(
    name = 'big5',
    model_num = model_num,
    size = 1000,
    param_names = param_names,
    latent_names = latent_names)
particles.set_log_dir(log_dir)
if gen_model:
    particles.compile_prior_model()
    particles.compile_model()
else:
    particles.load_prior_model()
    particles.load_model()

particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0
log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5
for t in tqdm(range(exp_data.size)):
    particles.get_incremental_weights(
        exp_data.get_stan_data_at_t(t)
        )
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    particles.update_weights()
    
    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
        particles.resample_particles()
        particles.jitter(exp_data.get_stan_data_upto_t(t+1))
        particles.reset_weights()
    else:
        particles.update_weights()

save_obj(log_lklhds, 'log_lklhds', log_dir)

100%|██████████| 50/50 [1:50:26<00:00, 132.53s/it]


In [8]:

print('\n\n')
marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.5f'%marg_lklhd)

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))




Marginal Likelihood 0.00000


Estimate
[ 0.02 -0.18  0.09  0.01  0.21  0.16 -0.08 -0.05 -0.04 -0.04 -0.2  -0.11
  0.13  0.1   0.09]


Estimate
[[ 0.58  0.2   0.19  0.07  0.05  0.11  0.05  0.09  0.04 -0.04 -0.04 -0.1
   0.06  0.09  0.08]
 [ 0.2   1.12  0.33  0.13  0.1   0.19  0.09  0.17  0.08 -0.08 -0.07 -0.18
   0.12  0.16  0.15]
 [ 0.19  0.33  0.88  0.12  0.09  0.19  0.09  0.16  0.08 -0.07 -0.07 -0.16
   0.1   0.14  0.13]
 [ 0.07  0.13  0.12  0.93  0.1   0.23  0.05  0.08  0.04 -0.05 -0.05 -0.12
   0.09  0.12  0.11]
 [ 0.05  0.1   0.09  0.1   0.87  0.15  0.03  0.06  0.03 -0.04 -0.04 -0.09
   0.07  0.09  0.08]
 [ 0.11  0.19  0.19  0.23  0.15  0.98  0.07  0.12  0.06 -0.08 -0.08 -0.2
   0.14  0.18  0.17]
 [ 0.05  0.09  0.09  0.05  0.03  0.07  0.73  0.32  0.15 -0.06 -0.06 -0.14
   0.09  0.11  0.11]
 [ 0.09  0.17  0.16  0.08  0.06  0.12  0.32  0.92  0.27 -0.11 -0.12 -0.26
   0.15  0.19  0.19]
 [ 0.04  0.08  0.08  0.04  0.03  0.06  0.15  0.27  1.13 -0.05 -0.05 -0.11
   0.07  0.09  0.09]
 

In [9]:
marg_lklhd

1.3916996226010373e-06